In [190]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [191]:
import random

import gym
import numpy as np

from gym import Env
from gym.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete, Tuple
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

<IPython.core.display.Javascript object>

In [429]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=-100, high=100, shape=(1,))
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        self.shower_length = 60

    def step(self, action):
        # apply temperature adjustment
        self.state += action - 1

        # decrease shower time
        self.shower_length -= 1

        # Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.shower_length = 60
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        return self.state


env = ShowerEnv()

<IPython.core.display.Javascript object>

In [427]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward

    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: 14
Episode: 2, Score: -60
Episode: 3, Score: -34
Episode: 4, Score: -22
Episode: 5, Score: -60


<IPython.core.display.Javascript object>

# Train model

In [428]:
env.reset()
model = PPO("MlpPolicy", env=env, verbose=1)
model.learn(total_timesteps=20000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -24.9    |
| time/              |          |
|    fps             | 2061     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.9       |
| time/                   |             |
|    fps                  | 1367        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012462255 |
|    clip_fraction        | 0.0656      |
|    clip_range           | 0.2         |
|    entropy_loss   

<IPython.core.display.Javascript object>

In [446]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-24.0, 54.99090833947008)

<IPython.core.display.Javascript object>